In [1]:
import torch
import math
torch.set_grad_enabled(False)

In [2]:
class Module(object):
    def forward (self, *input):
        raise NotImplementedError
        
    def backward ( self , * gradwrtoutput ) :
        raise NotImplementedError
    
    def param ( self ) :
        return []   
    
    def sigma(x):
        raise NotImplementedError
        
    def dsigma():
        raise NotImplementedError

In [3]:
class Losses(object):        
    def function():
        return NotImplementedError
    def derivative():
        NotImplementedError

In [4]:
class Optimizers(object):
    def step():
        return NotImplementedError

In [5]:
class Linear(Module):
    
    def __init__(self, input_dim, out_dim, bias = True):
        super().__init__()
        std = 1/math.sqrt(input_dim)
        
        #both initialized in [-std, std] with uniform pdf
        self.weights = torch.rand(out_dim, input_dim)
        self.weights = 2*std*self.weights - std
        
        if bias :
            self.bias = torch.rand(out_dim)
            self.bias = 2*std*self.bias - std
              
    def forward(self, x):
        self.x = x
        self.s = self.weights.mv(x) + self.bias
        self.o = self.sigma(self.s)
        return self.o
    
    def backward(self, loss, t, dl_dw, dl_db ):
        dl_dx = loss.derivative(self.o, t)
        dl_ds = self.dsigma(self.s) * dl_dx
        
        dl_dw.add_(dl_ds.view(-1, 1).mm(self.x.view(1, -1)))
        dl_db.add_(dl_ds)
        
    def sigma(self, x):
        return x

    def dsigma(self, x):
        return torch.ones_like(x)


In [ ]:
class ReLu(Module):
    
    def forward (self):
        raise NotImplementedError
        
    def backward ( self , * gradwrtoutput ) :
        raise NotImplementedError
    
    def param ( self ) :
        return []   
    
    def sigma(x):
        raise NotImplementedError
        
    def dsigma():
        raise NotImplementedError

In [6]:
class MSE(Losses):
    def function(self, v, t):
        return (v - t).pow(2).mean()
    
    def derivative(self, v, t):
        return 2 * (v - t)

In [7]:
linear = Linear(5, 6, True)
x = torch.randn(5)
b = torch.randn(5)
y = torch.randn(6)
linear.forward(x), linear.weights@x + linear.bias, MSE().function(x, b), torch.nn.MSELoss()(x, b)

(tensor([ 0.5119,  0.7017,  0.0324,  1.0508, -1.1644, -0.1763]),
 tensor([ 0.5119,  0.7017,  0.0324,  1.0508, -1.1644, -0.1763]),
 tensor(1.8435),
 tensor(1.8435))

In [8]:
dl_dw, dl_db = torch.zeros_like(linear.weights), torch.zeros_like(linear.bias)
linear.backward(MSE(),y, dl_dw, dl_db )

In [9]:
dl_dw

tensor([[ 0.1970,  0.2670, -1.2174,  2.3959,  0.9618],
        [ 0.0624,  0.0846, -0.3857,  0.7591,  0.3047],
        [-0.3623, -0.4911,  2.2392, -4.4068, -1.7691],
        [ 0.6006,  0.8141, -3.7119,  7.3050,  2.9325],
        [-0.4904, -0.6647,  3.0307, -5.9645, -2.3944],
        [-0.4987, -0.6760,  3.0824, -6.0662, -2.4352]])

In [10]:
dl_db

tensor([ 1.1062,  0.3505, -2.0346,  3.3727, -2.7538, -2.8008])

In [11]:
#TODO: check that the results is the same using builtin .backward()